# Esercizio 3 - SemEval

## Consegna

#### Parte 1

1. **Annotazione:** Annotare il file con i valori di similarità dati da noi e con i `BabelNetSynsetID` scelti da noi dal file `SemEval17_IT_senses2synsets.txt`.
   
2. **Agreement:** Dopo di che si calcola il `valore medio` di similarità, `Pearson` e `Spearman`
   
3. **Ricerca Automatica del Synset:** Andiamo a cercare il `BabelNetSynsetID` nel file dato dal prof `mini_NASARI.tsv`
   andando a massimizzare la *cosine similarity*. I valori che massimizzeranno quest'ultima saranno i nostri concetti (Synset).

   Per far ciò andiamo ad estrarre tutti i synsets relativi ad un termine, dal file `SemEval17_IT_senses2synsets.txt`
   e andiamo a calcolare la *cosine similarity* per ogni synset (quelli dei due termini) presente nel file `mini_NASARI.tsv`
   e salviamo i due sysnets che massimizzano la *cosine similarity*.

4. **Valutazione Similarità:** Sempre con `Pearson` e `Spearman`, tra i nostri valori di similarità e quelli trovati
   al passo precedente.

#### Parte 2

1. Associare i synset ai termini che pensiamo che massimizzino la similarità (fatto a mano) --> Gia fatto al passo 1 

2. Output costituito da 6 campi: 
   1. Term1
   2. Term2
   3. BS1 (dato da noi)
   4. BS2 (dato da noi)
   5. Sinonimi Term1 (da synset a lemma)
   6. Sinonimi Term2 (da synset a lemma)

   <br/>     
3. **Agreement dell'anotazione Synset trovati:** Tramite il punteggio `kappa di Cohen` calcoliamo il livello 
   di agreement (sklearn) tra i synset trovati da noi

4. **Valutazione annotazione:** Confronto i nostri concetti (synset) con quelli che massimizzano la `cos_sim`
   (Trovati già al passo 3)


## Sviluppo

#### Imports

In [1]:
import hashlib
import matplotlib.pyplot as plt

import re
import csv

from scipy import spatial

import numpy as np

### 1,2 - Annotazione e Agreement

#### Organizzazione dei dati

Andiamo a selezionare il dati da utilizzare per l'esercizio 3 tramite la funzione data dal professore

In [13]:
path_data = "../data/es3_res/it.test.data.txt"
annotator = "Bonicco"
data_file = "SemEval_data_" + annotator + ".tsv"

In [11]:
def get_range(surname):
    nof_elements = 500
    base_idx = (abs(int(hashlib.sha512(surname.encode('utf-8')).hexdigest(), 16)) % 10)
    idx_intervallo = base_idx * 50+1
    return idx_intervallo
 

input_name = "Bonicco"

values = []
sx = get_range(input_name)
values.append(sx)
dx = sx+50-1
intervallo = "" + str(sx) + "-" + str(dx)
print('{:15}:\tcoppie nell\'intervallo {}'.format(input_name, intervallo))

Bonicco        :	coppie nell'intervallo 51-100


#### Creazione file con le coppie di termini che ci interessano - **Non eseguire, file già creato**

In [14]:
with open(path_data) as file:
    lines = file.readlines()[50:100]
    lines = [line.rstrip() for line in lines]
    print(lines)

['biotopo\tbiologia', 'magma\tvulcano', 'brainstorming\ttelescopio', 'livello\tpunteggio', 'centesimo\taffare', 'partito politico\tassociazione', 'tsunami\tmare', 'struzzo\tfrutteto', 'cannella\tcaramella', 'scopa\tpolvere', 'galassia\tastronomo', 'succo\tfrappè', 'tapparella\ttenda', 'criminale\tcolpevole', 'cancro al pancreas\tchemioterapia', 'passato\tantecedente', 'Nazioni Unite\tParlamento Europeo', 'canzone\tesecutore', 'pistola\ttaccuino', 'acetilcolina\tiride', 'alfabeto\tpenna', 'coro\tcantante', 'legno\tcoperta', 'soldato\tpace', 'inglese\tamericano', 'lucertola\tcoccodrillo', 'denaro\tcontante', 'Polpo Paul\tpolpo', 'calendario\tvacanza', 'base\tsostanza chimica', 'governo\tlucertola', 'JPY\triciclaggio di denaro', 'DeepMind\tGoogle DeepMind', 'scacchi\tscacco al re', 'sonetto\tbellezza', 'lavoratore\tufficio', 'valuta\tscambio', 'poliestere\tcotone', 'tribunale\tgiustizia', 'coda\tretro', 'incidente\tlibro', 'groppo\tvento', 'piano\tarmonia', 'sultano\tministero', 'deserto\

#### Excel

Nel seguente file excel abbiamo annotato i punteggi di similarità relativi alla coppia di termini, dopo di che abbiamo calcolato la media tra i valori dati
dai due annotatori, oltre ai due indici di correlazione *Pearson* e *Spearman*.

Inoltre, abbiamo anche annotato i synset relativi ai termini, andandoli a scegliere tra la lista dei synset data dal professore.

<a href="https://1drv.ms/x/s!AkdbdBkdl_1dhbcgYl1tXDNpjknAIw?e=UYsHF5"> Link all'excel con i dati </a>

#### Calcolo Spearman per Excel

In [7]:
def pearson_index_np(x, y): # [3,45 ; 5,66 ; ..]
    '''
    Implementation of the Pearson index using numpy
    Args:
         x: golden value
         y: similarity list
    Returns:
        Pearson correlation index = [Covariance / (Standard deviation of x * Standard deviation of y)]
    '''
    mu_x = np.mean(x)
    mu_y = np.mean(y)
    
    # and the standard deviation of both
    std_dev_x = np.std(x)
    std_dev_y = np.std(y)
    
    numeric = std_dev_x * std_dev_y

    num = np.cov(x, y)[0][1] # Covariance    

    return num / numeric

def spearman_index(x, y):
    '''
    Implementation of the Spearman index.
    Args:
        x: golden value
        y: similarity list
    Returns:
         Spearman correlation index
    '''
    rank__x = define_rank(x)
    rank__y = define_rank(y)

    return pearson_index_np(rank__x, rank__y)


def define_rank(vector):
    '''
    Args:
        vector: numeric vector
    Returns:
        ranks list, sorted as the input order
    '''
    x_couple = [(vector[i], i) for i in range(len(vector))]
    x_couple_sorted = sorted(x_couple, key=lambda x: x[0])

    return [y for (x, y) in x_couple_sorted]

### da togleire

In [8]:
sim_paolo = [2,2.6,0,1.8,0.8,1.5,3,0,1,2.5,2.8,1,2.4,2.8,2.2,4,1.4,1.6,0,0.2,0.5,3,0,1.8,2,2.6,3,3,1,3,0,1.4,4,3.6,0.4,3,2.5,2,2.8,0.4,0,3,2.8,1.8,3.4,3.6,1.8,0,0,2.4]
sim_davide = [1.8,2,0,1.8,1.2,2.8,1,0,0.8,1,1,3,1.5,2.8,1.8,3.5,0.8,0,0,0.5,1,3,0.8,1,3.2,3,4,2.5,1,2,0,1.2,4,2,0,2,2,3,2,3.8,0,2,1.5,1.2,2,3,1.5,0,0,1.2]

print(len(sim_paolo) == len(sim_davide) == 50)

# sim_mean = [1.9, 2.3, 0, 1.8, 1, 2.15, 2, 0, 0.9, 1.75, 1.9, 2, 1.95, 2.8, 2, 3.75, 1.1, 0.8, 0, 0.35, 0.75, 3, 0.4, 1.4, 2.6, 2.8, 3.5, 2.75, 1, 2.5, 0, 1.3, 4, 2.8, 0.2, 2.5, 2.25, 2.5, 2.4, 2.1, 0, 2.5, 2.15,1.5, 2.7, 3.3, 1.65, 0, 0, 1.8]

True


In [9]:
spearman_index(sim_paolo, sim_davide)

0.1991817135017272

### Ricerca automatica del Synset - Massimizzazione cosine similarity da mini_NASARI

In [15]:
# open(data_file, 'w').close() # Puliamo il file con i termini di riferimento

In [16]:
terms_sim_score = []

with open(data_file, 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["Term1", "Term2", "Similarity"])
    
    for tuple in lines:
        w_couple = re.split(r'\t+', tuple.rstrip('\t'))
        print(w_couple)
        #! Scommentare per annotazione "automatica"
        # score = input("\tInserisci il valore di similarità [0 4] per i * "+ str(w_couple) +" *:")
        
        tsv_writer.writerow(w_couple)


['biotopo', 'biologia']
['magma', 'vulcano']
['brainstorming', 'telescopio']
['livello', 'punteggio']
['centesimo', 'affare']
['partito politico', 'associazione']
['tsunami', 'mare']
['struzzo', 'frutteto']
['cannella', 'caramella']
['scopa', 'polvere']
['galassia', 'astronomo']
['succo', 'frappè']
['tapparella', 'tenda']
['criminale', 'colpevole']
['cancro al pancreas', 'chemioterapia']
['passato', 'antecedente']
['Nazioni Unite', 'Parlamento Europeo']
['canzone', 'esecutore']
['pistola', 'taccuino']
['acetilcolina', 'iride']
['alfabeto', 'penna']
['coro', 'cantante']
['legno', 'coperta']
['soldato', 'pace']
['inglese', 'americano']
['lucertola', 'coccodrillo']
['denaro', 'contante']
['Polpo Paul', 'polpo']
['calendario', 'vacanza']
['base', 'sostanza chimica']
['governo', 'lucertola']
['JPY', 'riciclaggio di denaro']
['DeepMind', 'Google DeepMind']
['scacchi', 'scacco al re']
['sonetto', 'bellezza']
['lavoratore', 'ufficio']
['valuta', 'scambio']
['poliestere', 'cotone']
['tribunale'

## Inizio Esercizio

Vogliamo trovare il valore massimo di similarità tra due liste di synset, appartenenti a due termini

### Lista di synset associati ad un termine

In [17]:
def cos_sim(vec1, vec2):
    '''
    Giben 2 vectors, return the cosine similarity between them
    '''
    return 1 - spatial.distance.cosine(vec1, vec2)

def get_synsets_from_word(path, word):
    '''Given the path of the file with the list of synset and a word return the list of synset of the word
    Args:
        path: path of the file with the list of synset
        word: word to search in the file
    Returns:
        list of synset of the word
    '''
    syns_list = []
    with open(path) as f:
        terms_synsets = f.readlines()
        for i in range(len(terms_synsets)):
            if word in terms_synsets[i]:
                index = i
                while(terms_synsets[index+1].find('#') == -1):
                    syns_list.append(terms_synsets[index+1].rstrip('\n'))
                    index += 1
                break
    return syns_list


In [65]:
def parse_mini_NASARI(path):
    '''Given the path of the nasari file it return a dict with the synset as key and the vector as value
    Args:
        path: path of the nasari file
    Returns:
        A dict with the synset as key and the vector as value
    '''
    term_list = {}
    with open(path) as fd:
        rd = csv.reader(fd, delimiter="\t", quotechar='"')
        for row in rd:
            synset= row[0]
            index = synset.index('_')
            synset = synset[:index]
            term_list[synset] = row[1:]
            
    return term_list

def max_nasari_similarity(w1, w2):
    '''Given 2 word return the max value of similarity based on NASARI vector space model
    The method search for synset in nasari file and if find a value evaluate the similarity
    Args:
        w1: first word
        w2: second word
    Returns:
        max value of similarity
    '''
    path_synsets = "../data/es3_res/SemEval17_IT_senses2synsets.txt"
    path_nasari = "../data/es3_res/mini_NASARI.tsv"
    nasari = parse_mini_NASARI(path_nasari) # Dict with synset as key
    synsets_w1 = get_synsets_from_word(path_synsets, w1)
    synsets_w2 = get_synsets_from_word(path_synsets, w2)
    max_sim = [0, "", ""]
    for s1 in synsets_w1:
        if(s1 in nasari.keys()):
            for s2 in synsets_w2:
                if(s2 in nasari.keys()):
                    v1 = list(map(float, nasari[s1]))
                    v2 = list(map(float, nasari[s2]))
                    sim = cos_sim(v1, v2)
                    if sim > max_sim[0]:
                        max_sim = [sim, s1, s2]
    return max_sim

def couple_list(path):
    ''' Given a path return a list of couples of terms '''
    with open(path) as f:
        lines = f.readlines()
        couple_list = []
        for couple in lines:
            tmp = couple.split("\t")
            tmp[1] = tmp[1].rstrip('\n')
            couple_list.append(tmp)
        
        return couple_list
    

In [63]:
term_list_path = "resource/term_list.txt"

couples = couple_list(term_list_path)

In [66]:
for couple in couples:
    w1 = couple[0]
    w2 = couple[1]
    nasari_sim = max_nasari_similarity(w1, w2)
    print("Similarity between " + w1 + " and " + w2 + " is " + str(nasari_sim[0]))

Similarity between biotopo and biologia is 0.7280350719764522
Similarity between magma and vulcano is 0.866337996384421
Similarity between brainstorming and telescopio is 0.3177461381124145
Similarity between livello and punteggio is 0.9636678559650353
Similarity between centesimo and affare is 0.5394193390194117
Similarity between partito politico and associazione is 0.7150458360569183
Similarity between tsunami and mare is 0.6039623301956638
Similarity between struzzo and frutteto is 0.4547827996210163
Similarity between cannella and caramella is 0.7942815235847358
Similarity between scopa and polvere is 0.7368595348796247
Similarity between galassia and astronomo is 0.762448434314831
Similarity between succo and frappè is 0.8121781192172036
Similarity between tapparella and tenda is 0.7207209826657325
Similarity between criminale and colpevole is 0.866294843667411
Similarity between cancro al pancreas and chemioterapia is 0.9077949409690469
Similarity between passato and antecedente